In [0]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *
from google.colab import files
import os,shutil
from PIL import Image
import numpy
import torchvision
from google.colab import drive
from pathlib import Path


# Modifying ResNet

In [0]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x




def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model




# Importing Data From Drive

In [0]:

drive.mount('/content/gdrive/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
gdrive_path = Path('/content/gdrive/My Drive/Datasets')

In [0]:
try:
    os.mkdir('data')
finally:
    os.chdir('data')
try:  
    os.mkdir('image')
    os.mkdir('crappy')
finally:
    path = Path(os.getcwd())
    path_hr = path/'image'
    path_lr = path/'crappy'

In [0]:
os.listdir(path_hr) == []

True

In [0]:
#importing sample
if os.listdir(path_hr) == []:
    for i in range(len(os.listdir(gdrive_path/'images_I32bits'))):
        shutil.copy(gdrive_path/'images_I32bits'/(os.listdir(gdrive_path/'images_I32bits')[i]),path_hr)
        shutil.copy(gdrive_path/'crappy_I32bits'/(os.listdir(gdrive_path/'crappy_I32bits')[i]),path_lr)
    

In [0]:
for i in range(len(os.listdir(path_lr))):
  im = PIL.Image.open(path_lr/os.listdir(path_lr)[i])
  im = im.resize((100,100))
  im.save(path_lr/os.listdir(path_lr)[i])

In [0]:

im = PIL.Image.open(path_lr.ls()[0])
im.size

# Generator Model

In [0]:
bs,size = 15,100


Inputing x

In [0]:

src = ImageImageList.from_folder(path_lr, convert_mode = 'I').random_split_by_pct(0.1, seed = 42)

Inputing y

In [0]:
def get_data(bs,size):
  data = src.label_from_func(lambda x : path_lr/x.name, convert_mode = 'I')
  data = data.transform(get_transforms(max_zoom = 2.), size = size, tfm_y = True)
  data= data.databunch(bs=bs).normalize(do_y = True)
  data.c = 1
  return data

In [0]:
data_gen = get_data(bs,size)

In [0]:
data_gen.show_batch

In [0]:
wd = 1e-3
y_range = (-3.,3.)
loss = MSELossFlat()

In [0]:
arch = resnet34

#Changed Resnet to accept 1 channel - ok

In [0]:
def create_gen_learner():
  return unet_learner(data_gen,
                     arch,
                     wd = wd,
                     blur = True,
                     norm_type = NormType.Weight,
                     self_attention = True,
                     y_range = y_range,
                     loss_func = loss,
                     pretrained = False)



In [0]:
learn_gen = create_gen_learner()

In [0]:
learn_gen.fit(15,10e-4)

In [0]:
learn_gen.save('gen_pre2')

# Saving Generated Images

In [0]:
learn_gen.load('gen_pre2');

In [0]:
name_gen = 'image_gen'
path_gen = path/name_gen
#path_gen = gdrive_path/name_gen

In [0]:
path_gen.mkdir(exist_ok = True)

In [0]:
def save_gen(dl):
  i = 0
  names = dl.dataset.items
  
  for b in dl:
    preds = learn_gen.pred_batch(batch = b, reconstruct = True)
    for o in preds:
      o.save(path_gen/names[i].name)
      i += 1

In [0]:
save_gen(data_gen.fix_dl)

In [0]:
PIL.Image.open(path_gen.ls()[0]).size

In [0]:
PIL.Image.open(path_hr.ls()[0]).size

In [0]:
PIL.Image.open(path_lr.ls()[0]).size

# Training Critic

In [0]:
import pdb

In [0]:
learn_gen = None
gc.collect()

In [0]:
def get_crit_data(classes, bs, size):
  src = ImageList.from_folder(path, include = classes, convert_mode = 'I' ).split_by_rand_pct(0.1, seed = 42)
  ll = src.label_from_folder(classes = classes)
  data = (ll.transform(get_transforms(max_zoom = 2.), size = size).databunch(bs = bs).normalize(([0.485],[0.229]))) #(imagenet_stats)
  data.c = 1
  return data

In [0]:
imagenet_stats

In [0]:
data_crit = get_crit_data([name_gen, 'image'], bs = bs, size = size)

In [0]:
loss_critic = AdaptiveLoss(nn.BCEWithLogitsLoss())
#loss_critic = nn.BCEWithLogitsLoss()

In [0]:
loss_critic

In [0]:
_conv_args = dict(leaky=0.2, norm_type=NormType.Spectral)

def _conv(ni:int, nf:int, ks:int=3, stride:int=1, **kwargs):
    return conv_layer(ni, nf, ks=ks, stride=stride, **_conv_args, **kwargs)


def gan_critic(n_channels:int=3, nf:int=128, n_blocks:int=3, p:int=0.15):
    "Critic to train a `GAN`."
    layers = [
        _conv(n_channels, nf, ks=4, stride=2),
        nn.Dropout2d(p/2),
        res_block(nf, dense=True,**_conv_args)]
    nf *= 2 # after dense block
    for i in range(n_blocks):
        layers += [
            nn.Dropout2d(p),
            _conv(nf, nf*2, ks=4, stride=2, self_attention=(i==0))]
        nf *= 2
    layers += [
        _conv(nf, 1, ks=3, bias=False, padding=0, use_activ=False),
        Flatten()]
    return nn.Sequential(*layers)

In [0]:
def create_crit_learner(data, metrics):
  return Learner(data, gan_critic(n_channels = 1), metrics = metrics, loss_func = loss_critic, wd = wd)

In [0]:
learn_critic = create_crit_learner(data_crit, accuracy_thresh_expand) 
#learn_critic = create_crit_learner(data_crit, mean_absolute_error)

In [0]:
learn_critic.fit(15,10e-4)

Check Gan_critic() structure - last layer with wrong dimensions

In [0]:
learn_critic.save('critic-pre2')

# GAN

Now we combine the models into a GAN

In [0]:
learn_crit = None
learn_gen = None
gc.collect()

In [0]:
bs,size = 15,100


In [0]:
data_crit = get_crit_data([name_gen, 'image'], bs = bs, size = size)

In [0]:
learn_crit = create_crit_learner(data_crit, accuracy_thresh_expand).load(gdrive_path/'gan-crit')
#learn_crit = create_crit_learner(data_crit, metrics = None).load('critic-pre2')

In [0]:
learn_gen = create_gen_learner().load(gdrive_path/'gan-gen')
#learn_gen = create_gen_learner().load('gen_pre2')

In [0]:
switcher = partial(FixedGANSwitcher, n_crit = 10 , n_gen = 40)
#switcher = partial(AdaptiveGANSwitcher, gen_thresh = 2 , critic_thresh = 0.9)

learn = GANLearner.from_learners(learn_gen, learn_crit, weights_gen=(1., 50.), show_img = False, switcher = switcher, 
                                 opt_func = partial(optim.Adam, betas=(0.,0.99)), wd = wd)

learn.callback_fns.append(partial(GANDiscriminativeLR, mult_lr = 5.))

In [0]:
lr = 1e-5

In [0]:
#learn_gen.unfreeze()

In [0]:
learn.fit(5,lr)

In [0]:
learn_crit.save(gdrive_path/'gan-crit')
learn_gen.save(gdrive_path/'gan-gen')

In [0]:
learn.data = get_data(16,192)

In [0]:
learn.fit(10,lr/2)

In [0]:
learn.show_results(rows = 5)

In [0]:
data_gen

In [0]:
learn.save('gan-lc')

In [0]:
def save_to_gdrive(dl):
  i = 0
  names = dl.dataset.items
  
  for b in dl:
    preds = learn_gen.pred_batch(batch = b, reconstruct = True)
    for o in preds:
      o.save(gdrive_path/name_gen/names[i].name)
      i += 1

In [0]:
save_to_gdrive(data_gen.fix_dl)

#Fin